In [17]:
import pandas as pd
import random
from datetime import datetime

In [18]:
# 读取 Excel 文件
excel_file = pd.ExcelFile('../input_xlsx/data_template_v1.2.xlsx')

# 通过工作表名称读取
config_sheetname = "配置信息"
depots_sheetname = "仓库信息"
vehicleType_sheetname = "车型车辆信息"
orders_sheetname = "订单信息"
customer_sheetname = "客户信息"
tariff_sheetname = "承运商费率合同信息"

# config信息

| 参数名                          | 含义                                         | 类型    | 举例  | 默认值 | 是否必须 |
|------------------------------|--------------------------------------------|-------|-----|-----|------|
| config                       | 可选配置字段                                    | object| -   | -   | 是    |
| &emsp;DELIVERY_LOADING_SPEED  | 车辆装卸货速度（货量/分钟）                          | float | 1.5 | 无   | 是    |
| &emsp;DELIVERY_STAY_MINUTE     | 最少停留时间（分钟）                               | float | 8.0 | 无   | 是    |
| &emsp;ROS_VEHICLE_SPEED        | 车速（km/h）                                  | float | 50.0| 无   | 是    |
| &emsp;ROS_SPLIT_ORDER          | 是否允许拆单，"Y"表示允许，"N"表示不允许                 | string| Y   | 无   | 是    |
| &emsp;ROS_NORMAL_TEMPERATURE_RATIO | 冷藏车中常温订单最大占比                          | float | 0.90| 无   | 是    |


In [19]:
config_df = excel_file.parse(config_sheetname)

config_df.head(5)

,车辆最少停留时间（分钟）,卸货速度（吨/小时）,车辆是否返厂,车速（千米/小时）
0,8,2,否,50


In [20]:
config_df["卸货速度（吨/小时）"] = config_df["卸货速度（吨/小时）"].astype("float")
config_df["车辆最少停留时间（分钟）"] = config_df["车辆最少停留时间（分钟）"].astype("float")
config_df["车速（千米/小时）"] = config_df["车速（千米/小时）"].astype("float")

In [128]:
config = dict()
config["DELIVERY_LOADING_SPEED"] = config_df["卸货速度（吨/小时）"][0]//60
config["DELIVERY_STAY_MINUTE"] = config_df["车辆最少停留时间（分钟）"][0]
config["ROS_VEHICLE_SPEED"] = config_df["车速（千米/小时）"][0]
config["ROS_SPLIT_ORDER"] = "Y"
config["ROS_NORMAL_TEMPERATURE_RATIO"] = 0.99

In [129]:
config

{'DELIVERY_LOADING_SPEED': 0.0,
 'DELIVERY_STAY_MINUTE': 8.0,
 'ROS_VEHICLE_SPEED': 50.0,
 'ROS_SPLIT_ORDER': 'Y',
 'ROS_NORMAL_TEMPERATURE_RATIO': 0.99}

# depots信息


| 参数名  | 含义 | 类型 | 举例 | 默认值 | 是否必须 |
|---------|------|------|------|--------|----------|
| depots  | 物流中心。现行程序将只使用第一个物流中心，其余物流中心将被忽略。 | array[object] | - | - | 是       |
| &emsp;branchId     | 物流中心代码         | string  | WH01         | 无  | 是  |
| &emsp;branchDescr  | 物流中心名称         | string  | 张江物流中心 | 无  | 是  |
| &emsp;longitude    | 经度(东经为正，西经为负) | string  | 121.47390    | 无  | 是  |
| &emsp;latitude     | 纬度（北纬为正-南纬为负） | string  | 31.22805     | 无  | 是  |
| &emsp;notes        | 说明                 | string  | 仓库说明     | 无  | 是  |
| &emsp;branch_status| 仓库是否可用，"1"代表可用，"0"代表不可用 | string | 1            | 无  | 是  |


In [22]:
depots_df = excel_file.parse(depots_sheetname)

depots_df.head(5)

,仓库名称,仓库地址,地址编号,仓库经纬度
0,永盛一期工厂,鹿泉区石铜路36号,R005,"114.39982,37.974305"


In [23]:
depots_df['仓库名称'] = depots_df['仓库名称'].astype("str")
depots_df['仓库地址'] = depots_df['仓库地址'].astype("str")
depots_df['地址编号'] = depots_df['地址编号'].astype("str")
depots_df['仓库经纬度'] = depots_df['仓库经纬度'].astype("str")

# 使用 str.split() 方法拆分
depots_df[['longitude', 'latitude']] = depots_df['仓库经纬度'].str.split(',', expand=True)

In [24]:
depots_list = list()
for index, depot_info in depots_df.iterrows():
    depot = dict()
    depot["branchId"] = depot_info["地址编号"]
    depot["branchDesr"] = depot_info["仓库名称"]
    depot["longitude"] = depot_info["longitude"]
    depot["latitude"] = depot_info["latitude"]
    depot["notes"] = "note"
    depot["brach_status"] = "1"
    depots_list.append(depot)

# dispatchZone信息

## vehicleType

In [25]:
normal_vehicleType_df = excel_file.parse(vehicleType_sheetname)
Refrigerate_vehicleType_df = excel_file.parse(vehicleType_sheetname)
normal_vehicleType_df.head(5)

,车辆编号,所属承运商,车辆类型,最小载重（吨）,最大载重（吨）,最大容量（立方米）,车辆温控类型
0,NaN,NaN,9.6米,10,15.0,43,NaN
1,NaN,NaN,6.8米,6,9.5,28,NaN
2,NaN,NaN,4.2米,2,6.0,15,NaN
3,NaN,NaN,15米,20,25.0,63,NaN


In [26]:
normal_vehicleType_df["车辆温控类型"] = "常温"
Refrigerate_vehicleType_df["车辆温控类型"] = "冷藏"
vehicleType_df = pd.concat([normal_vehicleType_df, Refrigerate_vehicleType_df], ignore_index=True)
vehicleType_df["车型编号"] = ['Type' + str(index+1) for index in vehicleType_df.index]

In [27]:
vehicleType_df["车辆类型"] = vehicleType_df["车辆类型"].astype("str")
vehicleType_df["最小载重（吨）"] = vehicleType_df["最小载重（吨）"].astype("float")
vehicleType_df["最大载重（吨）"] = vehicleType_df["最大载重（吨）"].astype("float")
vehicleType_df["最大容量（立方米）"] = vehicleType_df["最大容量（立方米）"].astype("float")

In [28]:
vehicleType_list = list()
for index, vehicleType_info in vehicleType_df.iterrows():
    vehicleType = dict()
    vehicleType["vehicleTypeId"] = vehicleType_info["车型编号"]
    vehicleType["vehicleTypeDescr1"] = vehicleType_info["车辆类型"]
    vehicleType["loadCapacityMin"] = vehicleType_info["最小载重（吨）"]
    vehicleType["loadCapacity"] = vehicleType_info["最大载重（吨）"]
    vehicleType["capacity"] = vehicleType_info["最大容量（立方米）"]
    vehicleType["temperatureType"] = vehicleType_info["车辆温控类型"]
    vehicleType_list.append(vehicleType)

In [64]:
vehicleType_list

[{'vehicleTypeId': 'Type1',
  'vehicleTypeDescr1': '9.6米',
  'loadCapacityMin': 10.0,
  'loadCapacity': 15.0,
  'capacity': 43.0,
  'temperatureType': '常温'},
 {'vehicleTypeId': 'Type2',
  'vehicleTypeDescr1': '6.8米',
  'loadCapacityMin': 6.0,
  'loadCapacity': 9.5,
  'capacity': 28.0,
  'temperatureType': '常温'},
 {'vehicleTypeId': 'Type3',
  'vehicleTypeDescr1': '4.2米',
  'loadCapacityMin': 2.0,
  'loadCapacity': 6.0,
  'capacity': 15.0,
  'temperatureType': '常温'},
 {'vehicleTypeId': 'Type4',
  'vehicleTypeDescr1': '15米',
  'loadCapacityMin': 20.0,
  'loadCapacity': 25.0,
  'capacity': 63.0,
  'temperatureType': '常温'},
 {'vehicleTypeId': 'Type5',
  'vehicleTypeDescr1': '9.6米',
  'loadCapacityMin': 10.0,
  'loadCapacity': 15.0,
  'capacity': 43.0,
  'temperatureType': '冷藏'},
 {'vehicleTypeId': 'Type6',
  'vehicleTypeDescr1': '6.8米',
  'loadCapacityMin': 6.0,
  'loadCapacity': 9.5,
  'capacity': 28.0,
  'temperatureType': '冷藏'},
 {'vehicleTypeId': 'Type7',
  'vehicleTypeDescr1': '4.2米',


## vheicle

In [29]:
# 定义山东车牌号的前缀
shandong_prefixes = ['鲁A', '鲁B', '鲁C', '鲁D', '鲁E', '鲁F', '鲁G', '鲁H', '鲁J', '鲁K', '鲁L', '鲁M', '鲁N', '鲁P', '鲁Q', '鲁R', '鲁S', '鲁U', '鲁V', '鲁W', '鲁X', '鲁Y']
# 初始化车辆 ID 起始编号
vehicle_id_start = 1
# 定义车辆类型列表
vehicle_types = [f'Type{i}' for i in range(1, 9)]
# 初始化结果列表
vehicle_list = []
# 为每种车辆类型生成 50 个车辆信息
for vehicle_type in vehicle_types:
    for _ in range(50):
        # 生成车辆 ID
        vehicle_id = f"VE{str(vehicle_id_start).zfill(10)}"
        vehicle_id_start += 1
        # 随机选择一个山东车牌号前缀
        license_plate_prefix = random.choice(shandong_prefixes)
        # 生成随机的车牌号后五位（这里简单用数字表示）
        license_plate_suffix = ''.join(random.choices('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=5))
        # 组合成完整的车牌号
        license_plate_no = license_plate_prefix + license_plate_suffix
        # 创建车辆信息字典
        vehicle_info = {
            "vehicleId": vehicle_id,
            "licensePlateNo": license_plate_no,
            "vehicleType": vehicle_type
        }
        # 将车辆信息字典添加到结果列表中
        vehicle_list.append(vehicle_info)

## tariff

In [30]:
normal_tariff_df = excel_file.parse(tariff_sheetname)
Refrigerate_tariff_df = excel_file.parse(tariff_sheetname)
normal_tariff_df.head(5)

,项目,项目名称,线路,计价单位,结算点,线路编码,里程,冷藏报价,非冷藏报价,物流商,合同协议编码,补充协议编码,合同开始时间,合同结束时间
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,液奶干线,石家庄工厂发山东区域,石家庄-桓台,元/吨.公里,桓台,D02465,399.0,0.65,0.59,北京澳德物流有限责任公司,JLB-JT-202410-00098,NaN,2024-10-10,2026-09-30
2,NaN,NaN,石家庄-桓台-青州,元/吨.公里,青州,D02466,465.0,0.65,0.59,北京澳德物流有限责任公司,JLB-JT-202410-00098,NaN,2024-10-10,2026-09-30
3,NaN,NaN,石家庄-桓台-青州-安丘,元/吨.公里,安丘,D02467,551.0,0.60,0.54,北京澳德物流有限责任公司,JLB-JT-202410-00098,NaN,2024-10-10,2026-09-30
4,NaN,NaN,石家庄-桓台-青州-安丘-高密,元/吨.公里,高密,D02468,623.0,0.50,0.45,北京澳德物流有限责任公司,JLB-JT-202410-00098,NaN,2024-10-10,2026-09-30


In [31]:
normal_tariff_df["运输温控方式"] = "常温"
normal_tariff_df = normal_tariff_df.iloc[1:,:].drop('冷藏报价', axis=1).rename(columns={'非冷藏报价': '单价'})
Refrigerate_tariff_df["运输温控方式"] = "冷藏"
Refrigerate_tariff_df = Refrigerate_tariff_df.iloc[1:,:].drop('非冷藏报价', axis=1).rename(columns={'冷藏报价': '单价'})
tariff_df = pd.concat([normal_tariff_df, Refrigerate_tariff_df], ignore_index=True)
tariff_df["计价基准"] = tariff_df['计价单位'].apply(lambda x: '吨公里' if x == '元/吨.公里' else None)
tariff_df["始发地地点"] = tariff_df['线路'].apply(lambda x: x.split('-')[0])
tariff_df["目的地地点"] = tariff_df['线路'].apply(lambda x: x.split('-')[-1])

In [32]:
shipper_location_info = {
    "石家庄": {"shipperProvince": "河北省", "shipperCity": "石家庄市", "shipperDistrict": ""},
}
consignee_location_info = {
    "桓台": {"consigneeProvince": "山东省", "consigneeCity": "淄博市", "consigneeDistrict": "桓台县"},
    "青州": {"consigneeProvince": "山东省", "consigneeCity": "潍坊市", "consigneeDistrict": "青州市"},
    "安丘": {"consigneeProvince": "山东省", "consigneeCity": "潍坊市", "consigneeDistrict": "安丘市"},
    "高密": {"consigneeProvince": "山东省", "consigneeCity": "潍坊市", "consigneeDistrict": "高密市"},
    "惠民": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": "惠民县"},
    "高青": {"consigneeProvince": "山东省", "consigneeCity": "淄博市", "consigneeDistrict": "高青县"},
    "博兴": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": "博兴县"},
    "济阳": {"consigneeProvince": "山东省", "consigneeCity": "济南市", "consigneeDistrict": "济阳区"},
    "济南": {"consigneeProvince": "山东省", "consigneeCity": "济南市", "consigneeDistrict": ""},
    "章丘": {"consigneeProvince": "山东省", "consigneeCity": "济南市", "consigneeDistrict": "章丘区"},
    "莱州": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": "莱州市"},
    "招远": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": "招远市"},
    "龙口": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": "龙口市"},
    "蓬莱": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": "蓬莱区"},
    "乐陵": {"consigneeProvince": "山东省", "consigneeCity": "德州市", "consigneeDistrict": "乐陵市"},
    "即墨": {"consigneeProvince": "山东省", "consigneeCity": "青岛市", "consigneeDistrict": "即墨区"},
    "临清": {"consigneeProvince": "山东省", "consigneeCity": "聊城市", "consigneeDistrict": "临清市"},
    "聊城": {"consigneeProvince": "山东省", "consigneeCity": "聊城市", "consigneeDistrict": ""},
    "肥城": {"consigneeProvince": "山东省", "consigneeCity": "泰安市", "consigneeDistrict": "肥城市"},
    "新泰": {"consigneeProvince": "山东省", "consigneeCity": "泰安市", "consigneeDistrict": "新泰市"},
    "蒙阴": {"consigneeProvince": "山东省", "consigneeCity": "临沂市", "consigneeDistrict": "蒙阴县"},
    "青岛": {"consigneeProvince": "山东省", "consigneeCity": "青岛市", "consigneeDistrict": ""},
    "胶州": {"consigneeProvince": "山东省", "consigneeCity": "青岛市", "consigneeDistrict": "胶州市"},
    "黄岛": {"consigneeProvince": "山东省", "consigneeCity": "青岛市", "consigneeDistrict": "黄岛区"},
    "宁津": {"consigneeProvince": "山东省", "consigneeCity": "德州市", "consigneeDistrict": "宁津县"},
    "庆云": {"consigneeProvince": "山东省", "consigneeCity": "德州市", "consigneeDistrict": "庆云县"},
    "无棣": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": "无棣县"},
    "沾化": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": "沾化区"},
    "河口": {"consigneeProvince": "山东省", "consigneeCity": "东营市", "consigneeDistrict": "河口区"},
    "夏津": {"consigneeProvince": "山东省", "consigneeCity": "德州市", "consigneeDistrict": "夏津县"},
    "高唐": {"consigneeProvince": "山东省", "consigneeCity": "聊城市", "consigneeDistrict": "高唐县"},
    "茌平": {"consigneeProvince": "山东省", "consigneeCity": "聊城市", "consigneeDistrict": "茌平区"},
    "长清": {"consigneeProvince": "山东省", "consigneeCity": "济南市", "consigneeDistrict": "长清区"},
    "泰安": {"consigneeProvince": "山东省", "consigneeCity": "泰安市", "consigneeDistrict": ""},
    "莱芜": {"consigneeProvince": "山东省", "consigneeCity": "济南市", "consigneeDistrict": "莱芜区"},
    "阳信": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": "阳信县"},
    "滨州": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": ""},
    "利津": {"consigneeProvince": "山东省", "consigneeCity": "东营市", "consigneeDistrict": "利津县"},
    "广饶": {"consigneeProvince": "山东省", "consigneeCity": "东营市", "consigneeDistrict": "广饶县"},
    "东营": {"consigneeProvince": "山东省", "consigneeCity": "东营市", "consigneeDistrict": ""},
    "禹城": {"consigneeProvince": "山东省", "consigneeCity": "德州市", "consigneeDistrict": "禹城市"},
    "齐河": {"consigneeProvince": "山东省", "consigneeCity": "德州市", "consigneeDistrict": "齐河县"},
    "淄博": {"consigneeProvince": "山东省", "consigneeCity": "淄博市", "consigneeDistrict": ""},
    "寿光": {"consigneeProvince": "山东省", "consigneeCity": "潍坊市", "consigneeDistrict": "寿光市"},
    "潍坊": {"consigneeProvince": "山东省", "consigneeCity": "潍坊市", "consigneeDistrict": ""},
    "昌邑": {"consigneeProvince": "山东省", "consigneeCity": "潍坊市", "consigneeDistrict": "昌邑市"},
    "威海": {"consigneeProvince": "山东省", "consigneeCity": "威海市", "consigneeDistrict": ""},
    "烟台": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": ""},
    "淄川": {"consigneeProvince": "山东省", "consigneeCity": "淄博市", "consigneeDistrict": "淄川区"},
    "邹平": {"consigneeProvince": "山东省", "consigneeCity": "滨州市", "consigneeDistrict": "邹平市"},
    "平度": {"consigneeProvince": "山东省", "consigneeCity": "青岛市", "consigneeDistrict": "平度市"},
    "莱西": {"consigneeProvince": "山东省", "consigneeCity": "青岛市", "consigneeDistrict": "莱西市"},
    "莱阳": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": "莱阳市"},
    "海阳": {"consigneeProvince": "山东省", "consigneeCity": "烟台市", "consigneeDistrict": "海阳市"}
}

In [33]:
tariff_df["合同协议编码"] = tariff_df["合同协议编码"].astype("str")
tariff_df["线路"] = tariff_df["线路"].astype("str")
tariff_df["运输方式"] = "整车运输"
tariff_df["始发地地点"] = tariff_df['线路'].apply(lambda x: x.split('-')[0])
tariff_df["目的地地点"] = tariff_df['线路'].apply(lambda x: x.split('-')[-1])
tariff_df["运输温控方式"] = tariff_df["运输温控方式"].astype("str")
tariff_df["里程"] = tariff_df["里程"].astype("float")
tariff_df["计价基准"] = tariff_df["计价基准"].astype("str")
tariff_df["单价"] = tariff_df["单价"].astype("float")

In [34]:
tariff_list = list()
for index, tariff_info in tariff_df.iterrows():
    tariff = dict()
    tariff["tariffID"] = tariff_info["合同协议编码"]
    tariff["offeringType"] = "运输方式"
    tariff["shipperProvince"] = shipper_location_info[tariff_info["始发地地点"]]["shipperProvince"]
    tariff["shipperCity"] = shipper_location_info[tariff_info["始发地地点"]]["shipperCity"]
    tariff["shipperDistrict"] = shipper_location_info[tariff_info["始发地地点"]]["shipperDistrict"]
    tariff["consigneeProvince"] = consignee_location_info[tariff_info["目的地地点"]]["consigneeProvince"]
    tariff["consigneeCity"] = consignee_location_info[tariff_info["目的地地点"]]["consigneeCity"]
    tariff["consigneeDistrict"] = consignee_location_info[tariff_info["目的地地点"]]["consigneeDistrict"]
    tariff["temperatureType"] = tariff_info["运输温控方式"]
    tariff["mileage"] = tariff_info["里程"]
    tariff["rateBase"] = tariff_info["计价基准"]
    tariff["initialRate"] = tariff_info["单价"]
    tariff_list.append(tariff)

In [35]:
customer_df = excel_file.parse(customer_sheetname)
customer_df.head(5)

,客户编码,客户名称,客户等级,客户允许的可用车型,客户允许卸货的时间窗口,最长送达时间(小时),卸货周期（星期几可以收货）,发货波次
0,A001,低温阳信润丰,NaN,4.2米、6.8米、9.6米,8:00-18:00,14.154167,周一-周日,3：00-7：00
1,A002,低温惠民君莱宝,NaN,4.2米、6.8米、9.6米,8:00-18:00,16.654167,周一-周日,3：00-7：00
2,A003,滨州宝莱纳一鲜奶,NaN,4.2米、6.8米、9.6米,8:00-18:00,16.654167,周一-周日,3：00-7：00
3,A004,低温滨州宝莱纳,NaN,4.2米、6.8米、9.6米,8:00-18:00,16.654167,周一-周日,3：00-7：00
4,A005,低温滨州宝莱纳,NaN,4.2米、6.8米、9.6米,8:00-18:00,16.654167,周一-周日,3：00-7：00


In [36]:
customer_df["发货波次"] = customer_df["发货波次"].astype("str")
departureWave = ','.join(customer_df["发货波次"].unique())

In [50]:
dispatchZone_list = list()
dispatchZoneCodelist = ["DISZONE001"]
for dispathZoneCode in dispatchZoneCodelist:
    dispatchZone = dict()
    dispatchZone["dispatchZoneCode"] = dispathZoneCode
    dispatchZone["departureWave"] = departureWave
    dispatchZone["longitude"] = depots_list[0]["longitude"]
    dispatchZone["latitude"] = depots_list[0]["latitude"]
    dispatchZone["vehicleType"] = vehicleType_list
    dispatchZone["vehicle"] = vehicle_list
    dispatchZone["tariff"] = tariff_list
    dispatchZone_list.append(dispatchZone)

In [40]:
dispatchZone_list[0]

{'dispathZoneCode': 'DISZONE001',
 'departureWave': '3：00-7：00',
 'longitude': '114.39982',
 'latitude': '37.974305',
 'vehicleType': [{'vehicleTypeId': 'Type1',
   'vehicleTypeDescr1': '9.6米',
   'loadCapacityMin': 10.0,
   'loadCapacity': 15.0,
   'capacity': 43.0,
   'temperatureType': '常温'},
  {'vehicleTypeId': 'Type2',
   'vehicleTypeDescr1': '6.8米',
   'loadCapacityMin': 6.0,
   'loadCapacity': 9.5,
   'capacity': 28.0,
   'temperatureType': '常温'},
  {'vehicleTypeId': 'Type3',
   'vehicleTypeDescr1': '4.2米',
   'loadCapacityMin': 2.0,
   'loadCapacity': 6.0,
   'capacity': 15.0,
   'temperatureType': '常温'},
  {'vehicleTypeId': 'Type4',
   'vehicleTypeDescr1': '15米',
   'loadCapacityMin': 20.0,
   'loadCapacity': 25.0,
   'capacity': 63.0,
   'temperatureType': '常温'},
  {'vehicleTypeId': 'Type5',
   'vehicleTypeDescr1': '9.6米',
   'loadCapacityMin': 10.0,
   'loadCapacity': 15.0,
   'capacity': 43.0,
   'temperatureType': '冷藏'},
  {'vehicleTypeId': 'Type6',
   'vehicleTypeDescr1':

# orders信息

In [116]:
orders_df = excel_file.parse(orders_sheetname)
customer_df = excel_file.parse(customer_sheetname)
customer_df = customer_df.rename(columns={"客户编码":"客户编号","维度":"纬度"})
orders_df = orders_df.rename(columns={"维度":"纬度"})
orders_df.head(5)

,订单编号,客户编号,地址编号,收货省,收货市,收货区,目的地省市区详细地址,经度,纬度,温控类型,订单总重量（吨）,订单总体积（立方米）
0,812834867,A001,低温阳信润丰,山东省,阳信,NaN,山东省滨州市阳信县信城街道办事处劳家小庄村中段路西,117.435735,37.627133,冷藏,2.737,7.368
1,812835112,A002,低温惠民君莱宝,山东省,滨州,NaN,西20米 山东省滨州市滨城区北外环北海佲墅,117.985678,37.435452,冷藏,0.715,1.447
2,812836451,A003,滨州宝莱纳一鲜奶,山东省,滨州,NaN,西20米 山东省滨州市滨城区北外环北海佲墅,117.985678,37.435452,冷藏,0.679,1.275
3,812835124,A004,低温滨州宝莱纳,山东省,滨州,NaN,西20米 山东省滨州市滨城区北外环北海佲墅,117.985678,37.435452,冷藏,7.016,19.659
4,812835866,A005,低温滨州宝莱纳,山东省,滨州,NaN,西20米 山东省滨州市滨城区北外环北海佲墅,117.985678,37.435452,冷藏,0.004,0.004


In [117]:
order_c_df = pd.merge(orders_df, customer_df, on='客户编号', how='outer')

order_c_df["订单编号"] = order_c_df["订单编号"].astype("str")
order_c_df["创建时间"] = order_c_df.apply(lambda _: f"{datetime.now().strftime('%Y-%m-%d')}T{random.randint(0, 23):02d}:{random.randint(0, 59):02d}", axis=1)
order_c_df["温控类型"] = order_c_df["温控类型"].astype("str")
order_c_df['目的地省市区详细地址'] = order_c_df['目的地省市区详细地址'].astype("str")
encoded_values = pd.factorize(order_c_df['目的地省市区详细地址'])[0]

# 将编码值转换为指定格式
order_c_df["地址编码"] = ['DZ{:06d}'.format(val + 1) for val in encoded_values]

order_c_df["经度"] = order_c_df["经度"].astype("float")
order_c_df["纬度"] = order_c_df["纬度"].astype("float")
order_c_df["客户编号"] = order_c_df["客户编号"].astype("str")
order_c_df["客户允许的可用车型"] = order_c_df["客户允许的可用车型"].astype("str")
name_atr_to_code = {(vehicleType["vehicleTypeDescr1"],vehicleType["temperatureType"]): vehicleType["vehicleTypeId"] for vehicleType in vehicleType_list}

# 定义一个函数用于处理每行数据
def match_codes(row):
    names = row["客户允许的可用车型"].split('、')
    attrs = [row["温控类型"]]*len(names)
    codes = []
    for name, atr in zip(names, attrs):
        code = name_atr_to_code.get((name, atr))
        if code:
            codes.append(code)
    return ','.join(codes)
order_c_df["客户允许的可用车型编码"] = order_c_df.apply(match_codes, axis=1)
order_c_df["客户允许卸货的时间窗口"] = order_c_df["客户允许卸货的时间窗口"].astype("str")
order_c_df["最长送达时间(小时)"] = order_c_df["最长送达时间(小时)"].astype("float")
order_c_df["卸货周期（星期几可以收货）"] = order_c_df["卸货周期（星期几可以收货）"].astype("str")

In [118]:
order_c_df.head(1)

,订单编号,客户编号,地址编号,收货省,收货市,收货区,目的地省市区详细地址,经度,纬度,温控类型,...,客户名称,客户等级,客户允许的可用车型,客户允许卸货的时间窗口,最长送达时间(小时),卸货周期（星期几可以收货）,发货波次,创建时间,地址编码,客户允许的可用车型编码
0,812834867,A001,低温阳信润丰,山东省,阳信,NaN,山东省滨州市阳信县信城街道办事处劳家小庄村中段路西,117.435735,37.627133,冷藏,...,低温阳信润丰,NaN,4.2米、6.8米、9.6米,8:00-18:00,14.154167,周一-周日,3：00-7：00,2025-03-14T05:57,DZ000001,"Type7,Type6,Type5"


In [ ]:
def order_detail_generate(orderNo):
    orderNo="1"
    # 生成奶粉的 SKU 信息
    NF_sku_details_info = {}
    for i in range(1, 21):
        sku_code = f"NF{str(i).zfill(4)}"
        NF_sku_details_info[sku_code] = {
            "skuDescr1": "奶粉"+str(i),
            "weight": 1000,
            "cubic": 0.0017,
            "temperatureType": "常温",
            "grossWeight": 800,
            "cube": 0.0015
        }

    # 生成酸奶的 SKU 信息
    SN_sku_details_info = {}
    for i in range(1, 21):
        sku_code = f"NF{str(i).zfill(4)}"
        if i < 10:
            SN_sku_details_info[sku_code] = {
                "skuDescr1": "酸奶"+str(i),
                "weight": 320,
                "cubic": 0.0003,
                "temperatureType": "常温",
                "grossWeight": 300,
                "cube": 0.0003
            }
        elif 10 < i < 15:
            SN_sku_details_info[sku_code] = {
                "skuDescr1": "酸奶"+str(i),
                "weight": 182,
                "cubic": 0.00017,
                "temperatureType": "冷藏",
                "grossWeight": 180,
                "cube": 0.00017
            }
        elif i > 15:
            SN_sku_details_info[sku_code] = {
                "skuDescr1": "酸奶"+str(i),
                "weight": 205,
                "cubic": 0.00019,
                "temperatureType": "常温",
                "grossWeight": 200,
                "cube": 0.00019
            }

    # 合并奶粉和酸奶的 SKU 信息
    all_sku_details_info = {**NF_sku_details_info, **SN_sku_details_info}
    # 订单上限
    MAX_SKU_COUNT = 500
    MIN_WEIGHT = 400000
    MAX_WEIGHT = 3000000

    current_order = {}
    current_sku_count = 0
    current_weight = 0
    available_skus = list(all_sku_details_info.keys())

    while available_skus:
        # 随机选择一个 SKU
        sku = random.choice(available_skus)
        weight = int(all_sku_details_info[sku]["weight"])
        RANDOM_SKU_COUNT = random.randint(100, MAX_SKU_COUNT)
        # 检查加入该 SKU 后是否会超过 SKU 件数上限和总重量上限
        if current_sku_count <= RANDOM_SKU_COUNT:
            if sku in current_order:
                current_order[sku] += 1
            else:
                current_order[sku] = 1
            current_sku_count += 1
            current_weight += weight
        else:
            break
    order_detail_list = list()
    for index, (sku, count) in enumerate(current_order.items()):
        order_detail_list.append({"taskNo":orderNo, "taskLineNo": str(index+1),
                            "sku": sku, 
                            "skuDescr1": all_sku_details_info[sku]['skuDescr1'],
                            "qty": count,
                            "weight" : round(count*all_sku_details_info[sku]['weight']/ 1_000_000,5),
                            "cubic" : round(count*all_sku_details_info[sku]['cubic'],5),
                            "temperatureType" : all_sku_details_info[sku]['temperatureType'],
                            "grossWeight" : round(count*all_sku_details_info[sku]['grossWeight']/ 1_000_000,5),
                            "cube" : round(count*all_sku_details_info[sku]['cube'],5)
                            })
        qty = sum(sku["qty"] for sku in order_detail_list)
        weight =  sum(sku["weight"]*sku["qty"] for sku in order_detail_list)
        cubic = sum(sku["cubic"]*sku["qty"] for sku in order_detail_list)
    return qty, round(weight,3), round(cubic,3), order_detail_list

def get_time_complement(time_str):
    # 将时间字符串拆分成时间区间列表
    intervals = [interval.split('-') for interval in time_str.split(',')]
    # 初始化补集区间列表
    complement_intervals = []
    # 前一个区间的结束时间，初始为 00:00
    prev_end = '00:00'

    for start, end in intervals:
        if start > prev_end:
            # 如果当前区间的开始时间大于前一个区间的结束时间，说明存在空闲区间
            complement_intervals.append(f"{prev_end}-{start}")
        # 更新前一个区间的结束时间
        prev_end = end

    # 检查最后一个区间结束后到 24:00 是否有空闲区间
    if prev_end < '24:00':
        complement_intervals.append(f"{prev_end}-24:00")

    # 将补集区间列表转换为字符串
    return ','.join(complement_intervals)

In [120]:
order_c_df.columns

Index(['订单编号', '客户编号', '地址编号', '收货省', '收货市', '收货区', '目的地省市区详细地址', '经度', '纬度',
       '温控类型', '订单总重量（吨）', '订单总体积（立方米）', '客户名称', '客户等级', '客户允许的可用车型',
       '客户允许卸货的时间窗口', '最长送达时间(小时)', '卸货周期（星期几可以收货）', '发货波次', '创建时间', '地址编码',
       '客户允许的可用车型编码'],
      dtype='object')

In [121]:
orders_list = list()
for index, orders_info in order_c_df.iterrows():
    order = dict()
    order["dispatchZoneCode"] = dispatchZone_list[0]["dispatchZoneCode"]
    order["taskNo"] = orders_info["订单编号"]
    order["orderNo"] = orders_info["订单编号"]
    order["DC"] = depots_list[0]["branchId"]
    order["createTime"] = orders_info["创建时间"]
    order["routeNoNext"] = "",
    order["offeringType"] = "汽运"
    order["temperatureClass"] = orders_info['温控类型']
    order["addressNo"] = orders_info['地址编码']
    order["address"] = orders_info['目的地省市区详细地址']
    order["vehicleCategory"] = ','.join([vehicleType["vehicleTypeId"] for dispatchZone in dispatchZone_list if dispatchZone["dispatchZoneCode"] == dispatchZone_list[0]["dispatchZoneCode"] for vehicleType in dispatchZone["vehicleType"]])
    order["longitude"] = orders_info["经度"]
    order["latitude"] = orders_info["纬度"]
    order["consigneeId"] = orders_info["客户编号"]
    order["companyClass"] = ""
    order["availableVehicleType"] = orders_info["客户允许的可用车型编码"]
    order["vehicleCategoryCus"] = ','.join(set(order["vehicleCategory"].split(',')).intersection(set(order["availableVehicleType"].split(','))))
    order["unloadingTimeWindow"] = orders_info["客户允许卸货的时间窗口"]
    order["nonunloadingTimeWindow"] = get_time_complement(orders_info["客户允许卸货的时间窗口"])
    order["leadTime"] = orders_info['最长送达时间(小时)']
    datedic = {'周一': 1, '周二': 2, '周三': 3, '周四': 4, '周五': 5, '周六': 6, '周日': 7}
    order["SDD"] = (','.join(str(i) for i in range(datedic[orders_info['卸货周期（星期几可以收货）'].split('-')[0]], 
                                                   datedic[orders_info['卸货周期（星期几可以收货）'].split('-')[1]] + 1)))
    order["qty"], order["weight"], order["cubic"], order["order_detail_list"] = order_detail_generate(order["orderNo"])

In [122]:
order

{'dispatchZoneCode': 'DISZONE001',
 'taskNo': '802835386',
 'orderNo': '802835386',
 'DC': 'R005',
 'createTime': '2025-03-14T20:33',
 'routeNoNext': ('',),
 'offeringType': '汽运',
 'temperatureClass': '常温',
 'addressNo': 'DZ000036',
 'address': '路7号 山东省青岛市城阳区北部工业园祺阳',
 'vehicleCategory': 'Type1,Type2,Type3,Type4,Type5,Type6,Type7,Type8',
 'longitude': 120.240996610331,
 'latitude': 36.2604603611225,
 'consigneeId': 'A069',
 'companyClass': '',
 'availableVehicleType': 'Type3,Type2,Type1',
 'vehicleCategoryCus': 'Type3,Type1,Type2',
 'unloadingTimeWindow': '8:00-17:00',
 'nonunloadingTimeWindow': '00:00-8:00,17:00-24:00',
 'leadTime': 37.1911111111403,
 'SDD': '1,2,3,4,5,6,7',
 'qty': 115,
 'weight': 0.256,
 'cubic': 0.314,
 'order_detail_list': [{'taskNo': '1',
   'taskLineNo': '1',
   'sku': 'NF0004',
   'skuDescr1': '酸奶4',
   'qty': 8,
   'weight': 0.00256,
   'cubic': 0.0024,
   'temperatureType': '常温',
   'grossWeight': 0.0024,
   'cube': 0.0024},
  {'taskNo': '1',
   'taskLineNo':